### Import of data & packages

In [18]:
# import pandas for data cleaning and plotly for graphing
from plotly.subplots import make_subplots
import pandas as pd
import plotly.graph_objects as go

### Get census data

In [19]:
# read in census data - notably not encoded in utf-8
init = pd.read_csv('CensusPopData2019.csv', encoding = "ISO-8859-1")
init.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2019,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019
0,40,3,6,1,0,Alabama,Alabama,4779736,4780125,4785437,...,1.917501,0.578434,1.186314,1.522549,0.563489,0.626357,0.745172,1.090366,1.773786,2.483744
1,50,3,6,1,1,Alabama,Autauga County,54571,54597,54773,...,4.847310,6.018182,-6.226119,-3.902226,1.970443,-1.712875,4.777171,0.849656,0.540916,4.560062
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183112,...,24.017829,16.641870,17.488579,22.751474,20.184334,17.725964,21.279291,22.398256,24.727215,24.380567
3,50,3,6,1,5,Alabama,Barbour County,27457,27455,27327,...,-5.690302,0.292676,-6.897817,-8.132185,-5.140431,-15.724575,-18.238016,-24.998528,-8.754922,-5.165664
4,50,3,6,1,7,Alabama,Bibb County,22915,22915,22870,...,1.385134,-4.998356,-3.787545,-5.797999,1.331144,1.329817,-0.708717,-3.234669,-6.857092,1.831952


In [20]:
# need the list function or it only does the first and last few
list(init.columns)

['SUMLEV',
 'REGION',
 'DIVISION',
 'STATE',
 'COUNTY',
 'STNAME',
 'CTYNAME',
 'CENSUS2010POP',
 'ESTIMATESBASE2010',
 'POPESTIMATE2010',
 'POPESTIMATE2011',
 'POPESTIMATE2012',
 'POPESTIMATE2013',
 'POPESTIMATE2014',
 'POPESTIMATE2015',
 'POPESTIMATE2016',
 'POPESTIMATE2017',
 'POPESTIMATE2018',
 'POPESTIMATE2019',
 'NPOPCHG_2010',
 'NPOPCHG_2011',
 'NPOPCHG_2012',
 'NPOPCHG_2013',
 'NPOPCHG_2014',
 'NPOPCHG_2015',
 'NPOPCHG_2016',
 'NPOPCHG_2017',
 'NPOPCHG_2018',
 'NPOPCHG_2019',
 'BIRTHS2010',
 'BIRTHS2011',
 'BIRTHS2012',
 'BIRTHS2013',
 'BIRTHS2014',
 'BIRTHS2015',
 'BIRTHS2016',
 'BIRTHS2017',
 'BIRTHS2018',
 'BIRTHS2019',
 'DEATHS2010',
 'DEATHS2011',
 'DEATHS2012',
 'DEATHS2013',
 'DEATHS2014',
 'DEATHS2015',
 'DEATHS2016',
 'DEATHS2017',
 'DEATHS2018',
 'DEATHS2019',
 'NATURALINC2010',
 'NATURALINC2011',
 'NATURALINC2012',
 'NATURALINC2013',
 'NATURALINC2014',
 'NATURALINC2015',
 'NATURALINC2016',
 'NATURALINC2017',
 'NATURALINC2018',
 'NATURALINC2019',
 'INTERNATIONALMIG201

In [21]:
# sumlev=40 means only state-level data and not county-level
# this pulls pop of each state into a df that's a lot more manageable
state_pops = init[init['SUMLEV'] == 40][['STNAME','POPESTIMATE2019']]


### Get senator data

In [22]:
# pull every table from the wikipedia page
sen_scrape = pd.read_html('https://en.wikipedia.org/wiki/List_of_current_United_States_senators')

In [23]:
# pull the first table of length 100; unlikely that there would ever be a prior table with a hundred entries
for table in sen_scrape:
    if (len(table) == 100):
        sen_init = table
        break

# and pare it down to the columns we want
sen_init = sen_init[['State', 'Senator', 'Party.1', 'Born', 'Assumed office', 'Term up']]


state_abbrevs = pd.read_csv('kosal/States.csv')
state_abbrevs.head()

def myUpper(t):
    return t.upper()

sen_init['temp_upper'] = sen_init['State'].apply(myUpper)

sen_init = sen_init.merge(state_abbrevs, how='left', left_on= 'temp_upper', right_on = 'states')
# ANALYSIS NOTE: Occupation / previous office / residence possibly useful later?

### Combine data & clean

In [24]:
# merge census data with senate data by state to get a final dataframe of info
sen_with_pop = sen_init.merge(state_pops, how='left', left_on='State', right_on='STNAME')

# removes all footnotes (of the form [a]) with regex
sen_with_pop.replace("\[\w*\]","", inplace = True, regex = True)

# rename columns and remove duplicate
sen_with_pop.rename({'Party.1': 'party', 
                     'POPESTIMATE2019':'population', 
                     'Assumed office':'assumed_office',
                     'Term up': 'term_up'}, inplace = True, axis=1)
sen_with_pop.drop(['STNAME', 'states', 'temp_upper'], axis=1, inplace=True)
sen_with_pop

,State,Senator,party,Born,assumed_office,term_up,st,population
0,Alabama,Richard Shelby,Republican,(age 86),"January 3, 1987",2022,AL,4903185
1,Alabama,Tommy Tuberville,Republican,(age 66),"January 3, 2021",2026,AL,4903185
2,Alaska,Lisa Murkowski,Republican,(age 63),"December 20, 2002",2022,AK,731545
3,Alaska,Dan Sullivan,Republican,(age 56),"January 3, 2015",2026,AK,731545
4,Arizona,Kyrsten Sinema,Democratic,(age 44),"January 3, 2019",2024,AZ,7278717
...,...,...,...,...,...,...,...,...
95,West Virginia,Shelley Moore Capito,Republican,(age 67),"January 3, 2015",2026,WV,1792147
96,Wisconsin,Ron Johnson,Republican,(age 65),"January 3, 2011",2022,WI,5822434
97,Wisconsin,Tammy Baldwin,Democratic,(age 59),"January 3, 2013",2024,WI,5822434
98,Wyoming,John Barrasso,Republican,(age 68),"June 25, 2007",2024,WY,578759


In [25]:

#sen_with_pop.to_csv('SPopulation.csv', index=False) # Read into a csv so can pull from that later

In [26]:
more_sen = pd.read_csv('../SSSalamanders_RevA.csv')

select_more_sen = more_sen[['117th Senators', 'Religion', 'Gender', 'Birth Year']]
select_more_sen.head()

,117th Senators,Religion,Gender,Birth Year
0,Alex Padilla,Catholic,male,3/22/1973
1,Amy Klobuchar,Congregationalist,female,5/25/1960
2,Angus King,Episcopalian,male,3/31/1944
3,Ben Cardin,Jewish,male,10/5/1943
4,Ben Ray Luján,Catholic,male,6/7/1972


In [27]:
select_more_sen['Religion'].unique()

array(['Catholic', 'Congregationalist', 'Episcopalian', 'Jewish',
       'Presbyterian', 'Evangelical', 'Protestant', 'Baptist', 'Lutheran',
       'Methodist', 'Restorationist', 'Quaker', 'Holiness',
       'Unaffiliated', 'Buddhist', 'Unknown', 'Latter-day Saint'],
      dtype=object)

In [28]:
more_sen.columns

Index(['117th Senators', 'State', 'Party', 'Title', 'Religion', 'Gender',
       'Birth Year', 'Assumed Office', 'WSJ', '538', 'Am Conservative Union',
       'Abortion', 'immigrants' rights', 'filibuster', 'Right to Life'],
      dtype='object')

In [29]:
sen_data = sen_with_pop.merge(select_more_sen, how='left', left_on = 'Senator', right_on = '117th Senators')
sen_data.drop(['Gender', 'Birth Year', '117th Senators'], axis=1, inplace = True)
sen_data.head()

,State,Senator,party,Born,assumed_office,term_up,st,population,Religion
0,Alabama,Richard Shelby,Republican,(age 86),"January 3, 1987",2022,AL,4903185,Presbyterian
1,Alabama,Tommy Tuberville,Republican,(age 66),"January 3, 2021",2026,AL,4903185,Restorationist
2,Alaska,Lisa Murkowski,Republican,(age 63),"December 20, 2002",2022,AK,731545,Catholic
3,Alaska,Dan Sullivan,Republican,(age 56),"January 3, 2015",2026,AK,731545,Catholic
4,Arizona,Kyrsten Sinema,Democratic,(age 44),"January 3, 2019",2024,AZ,7278717,Unaffiliated


In [30]:
sen_names = sen_data['Senator']

#sen_genders = []

# for sen in sen_names: # 1 for male, 2 for female
#     currsex = input(f"{sen}: ")
#     sen_genders.append(currsex)

sen_genders = [1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 
 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1,2,1,1,1,1,1,1,1,
 1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,2,2,1,2,1,2,1,2]

In [31]:
sen_genders = [str(i) for i in sen_genders]


sen_data['gender'] = sen_genders
sen_sex_ratio = list(sen_data['gender'].value_counts())



In [32]:
# Gender graph 2: Chamber style
colordict = {'Republican': 'red', 'Democratic':'blue', 'Independent':'gray', 'Vacant':'purple'}
colors = [colordict[k] for k in list(sen_data['party'])]
sen_data['color'] = colors

# setup

sex_data = sen_data.sort_values(by='gender')


sex_colordict = {'1': 'purple', '2': 'pink'}
sex_colors = [sex_colordict[k] for k in list(sex_data['gender'])]
sex_data['sex_colors'] = sex_colors



# put coords in table?
rep_coords = [(r, a*90 / (r+2)) for a in range(12) for r in range(max(5, a-2),10)]
dem_coords = [(r, 180 - a*90 / (r+2)) for a in range(12) for r in range(max(5, a-2),10)]


Ds_sex = sex_data[sex_data['party'] != 'Republican']
Rs_sex = sex_data[sex_data['party'] == 'Republican']

Ds_sex.head()


,State,Senator,party,Born,assumed_office,term_up,st,population,Religion,gender,color,sex_colors
68,Ohio,Sherrod Brown,Democratic,(age 68),"January 3, 2007",2024,OH,11689100,Lutheran,1,blue,purple
62,New York,Chuck Schumer,Democratic,(age 70),"January 3, 1999",2022,NY,19453561,Jewish,1,blue,purple
61,New Mexico,Ben Ray Luján,Democratic,(age 48),"January 3, 2021",2026,NM,2096829,Catholic,1,blue,purple
60,New Mexico,Martin Heinrich,Democratic,(age 49),"January 3, 2013",2024,NM,2096829,Lutheran,1,blue,purple
59,New Jersey,Cory Booker,Democratic,(age 51),"October 31, 2013",2026,NJ,8882190,Baptist,1,blue,purple


In [33]:
sen_data['race'] = [1] * 100

def rerace(senator, n):
    sen_data.loc[sen_data['Senator'] == senator, ['race']] = n
    return "complete"

black = ["Cory Booker", 'Tim Scott', 'Raphael Warnock']
asian = ['Mazie Hirono', 'Tammy Duckworth']

[rerace(a, 2) for a in black]
[rerace(b, 4) for b in asian]

sen_data.to_csv('../resources/SPopulation.csv')

African Americans: Cory Booker, Tim Scott, Raphael Warnock
Asian Americans: Mazie Hirono, Tammy Duckworth
Hispanic Americans: Robert Menendez, Marco Rubio, Ted Cruz, Catherine Cortez Masto, Ben Ray Lujan, Alex Padilla
Source: https://www.senate.gov/senators/EthnicDiversityintheSenate.htm

1 = White Alone
2 = Black or African American Alone 
3 = American Indian or Alaska Native Alone
4 = Asian Alone

In [34]:
# NOTE: Should pull religion data from these tables: https://en.wikipedia.org/wiki/Religious_affiliation_in_the_United_States_Senate

